<a href="https://colab.research.google.com/github/PedroYutaroUEA/RNA_PP2.1/blob/main/RNA_P21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Importação das bibliotecas basicas.


In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import pandas as pd



Começar o projeto